<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M3_LSTM_stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series with LSTM

In this notebook we will be using an LSTM architecture to work with stock price (change) prediciton.
We will first look at simple 1-step ahead prediction, then at multi-step and finally at multi-step multi-feature prediction, and finally evaluate the performance of the model using a "naive backtesting" approach. Here the question is: How much money would we have gained/lost if we followed the predictions of the network.

In [ ]:
# First: Let's install the data-reader to get some current stock prices
!pip install --upgrade pandas-datareader -q

In [ ]:
# Import basic packaging

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pandas_datareader import data as pdr
import datetime as dt

import seaborn as sns
sns.set()

In [ ]:
# set timeframe for data extraction

start = dt.datetime(2021,1,1)
end = dt.datetime.now()

In [ ]:
# get the data

data =  pdr.DataReader("^OMXC25",'yahoo', start=start, end=end)

In [ ]:
data.info()

In [ ]:
# calculate diffs - changes to previous period
data_diff = data.diff()

In [ ]:
# alternatively
#data_diff = data.pct_change()

In [ ]:
data_diff

Here we will be using diffs to predict diffs ahead.

In [ ]:
# import packaging for Neural Net
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

We willpreprocess the data by MinMax scaling between -1 and 1

In [ ]:
# normalizing
scaler = MinMaxScaler(feature_range=(-1, 1))
data_diff['Adj_Close_scaled'] = scaler.fit_transform(data_diff['Adj Close'].values.reshape(-1, 1))

In [ ]:
# create targets by shifting
data_diff['Adj_Close_scaled+1'] = data_diff.Adj_Close_scaled.shift(-1, fill_value=data_diff.Adj_Close_scaled.iloc[-1])

In [ ]:
# get the data as matrix
data_p = data_diff.iloc[1:,6:].values.astype('float32')

This is how our dataframe looks. X is change in t; y is change in t+1

In [ ]:
data_p

We will use a manual split. We cannot shuffle anything...because time 😅

In [ ]:
# split into train and test sets
train_size = int(len(data_p) * 0.67)
test_size = len(data_p) - train_size

train, test = data_p[0:train_size,:], data_p[train_size:len(data_p),:]
print(len(train), len(test))

In [ ]:
X_train = train[:,0]
y_train = train[:,1]

X_test = test[:,0]
y_test = test[:,1]

To work with Keras, we need to bring data into a 3D (tensor) format
samples here means lenght of the series; time steps = how much do we look back; features: how many variables are there in our X?

In [ ]:
# reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, 1))
X_test = np.reshape(X_test, (X_test.shape[0], 1, 1))

In [ ]:
X_train.shape

In [ ]:
X_train

## 1-step LSTM

We start with a smile net with 4 LSTM cells. The output layer has 1 neuron (because it's a regression problem with one outcome)

In [ ]:
# build the network

model = Sequential()
model.add(LSTM(4, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
# let's fit the model
model.fit(X_train, y_train, epochs=20, batch_size=20, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform([y_train])

testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.4f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[0], testPredict[:,0]))
print('Test Score: %.4f RMSE' % (testScore))

In [ ]:
data_diff['Adj_close_pred'] = data_diff['Adj Close']

In [ ]:
data_diff['Adj_close_pred'].iloc[-testPredict.shape[0]:] = testPredict.flatten()

In [ ]:
data_diff.loc[:,['Adj Close','Adj_close_pred']].plot()

Our network is a Chicken...doesn't dare to make real change predictions

In [ ]:
# let's look at the sign for the "mini momevements" that are produced
pred_sign = pd.DataFrame(zip(testPredict.flatten(), y_test.flatten())) > 0

Basically: True if > 0, else False

In [ ]:
pred_sign

What's the share where the predicted sign (direction of change) was the same as reality?

In [ ]:
(pred_sign[0] == pred_sign[1]).sum()/len(pred_sign)

## Introducing multi-step

In [ ]:
# convert an array of values into a dataset matrix (convenience function to fiddle less around)
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t and Y=t+1
look_back = 5
X_train, y_train = create_dataset(train, look_back)
X_test, y_test = create_dataset(test, look_back)

In [ ]:
# build the network

model = Sequential()
model.add(LSTM(16, input_shape=(look_back, 1), return_sequences=True))
model.add(LSTM(4, return_sequences=False))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=1, verbose=2)

In [ ]:
# make predictions
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

In [ ]:
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
y_train = scaler.inverse_transform([y_train])

testPredict = scaler.inverse_transform(testPredict)
y_test = scaler.inverse_transform([y_test])

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
data_diff['Adj_close_pred'] = data_diff['Adj Close']

In [ ]:
data_diff['Adj_close_pred'].iloc[-testPredict.shape[0]:] = testPredict.flatten()

In [ ]:
data_diff.loc[:,['Adj_close_pred']].plot()
data_diff.loc[:,['Adj Close']].plot()

In [ ]:
pred_sign = pd.DataFrame(zip(testPredict.flatten(), y_test.flatten())) > 0
(pred_sign[0] == pred_sign[1]).sum()/len(pred_sign)

## Multi-step & Multi-feature
Let's use the built in time-series-generator for a bit more compelx case
We are going to predict ETH change from the scaled values of BTC, ADA, DOGE and BNB.

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
start = dt.datetime(2022,5,1)
end = dt.datetime.now()

In [ ]:
ETH =  pdr.DataReader('ETH-USD','yahoo', start=start, end=end)
BTC =  pdr.DataReader('BTC-USD','yahoo', start=start, end=end)
ADA =  pdr.DataReader('ADA-USD','yahoo', start=start, end=end)
DOGE =  pdr.DataReader('DOGE-USD','yahoo', start=start, end=end)
BNB =  pdr.DataReader('BNB-USD','yahoo', start=start, end=end)

In [ ]:
data = pd.DataFrame({'ETH':ETH['Adj Close'], 
                     'BTC':BTC['Adj Close'], 
                     'ADA':ADA['Adj Close'], 
                     'DOGE': DOGE['Adj Close'], 
                     'BNB':BNB['Adj Close']})

In [ ]:
data

In [ ]:
data = data.sort_index(ascending=True)

In [ ]:
data['ETH_shift'] = data['ETH'].shift(-1, fill_value=data['ETH'].iloc[-1])

In [ ]:
test_size = int(len(data) * 0.2) # the test data will be 20% (0.2) of the entire data
train = data.iloc[:-test_size,:].copy() 
# the copy() here is important, it will prevent us from getting: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame. Try using .loc[row_index,col_indexer] = value instead
test = data.iloc[-test_size:,:].copy()
print(train.shape, test.shape)

In [ ]:
train

In [ ]:
X_train = train.iloc[:,:5].values
y_train = train.iloc[:,5].values

X_test = test.iloc[:,:5].values
y_test = test.iloc[:,5].values

In [ ]:
x_scaler = MinMaxScaler(feature_range=(-1, 1))
y_scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
X_train = x_scaler.fit_transform(X_train)
y_train = y_scaler.fit_transform(y_train.reshape(-1,1))

X_test = x_scaler.transform(X_test)
y_test = y_scaler.transform(y_test.reshape(-1,1))

In [ ]:
n_input = 5 #how many samples/rows/timesteps to look in the past in order to forecast the next sample
n_features= X_train.shape[1] # how many predictors/Xs/features we have to predict y
b_size = 32 # Number of timeseries samples in each batch
generator = TimeseriesGenerator(X_train, y_train, length=n_input, batch_size=b_size)


In [ ]:
print(generator[0][0].shape)

In [ ]:
model = Sequential()
model.add(LSTM(150, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(generator,epochs=5)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch);

In [ ]:
test_generator = TimeseriesGenerator(X_test, np.zeros(len(X_test)), length=n_input, batch_size=b_size)
print(test_generator[0][0].shape)

In [ ]:
y_pred_scaled = model.predict(test_generator)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
y_test = y_scaler.inverse_transform(y_test)
results = pd.DataFrame({'y_true':y_test.flatten()[n_input:],'y_pred':y_pred.flatten()})
print(results)

In [ ]:
results.plot()

In [ ]:
pred_sign = results.diff(1) > 0

In [ ]:
pred_sign.y_true == pred_sign.y_pred

In [ ]:
(pred_sign.y_true == pred_sign.y_pred).sum()/len(pred_sign)

## From here some experimental backtesting...

In [ ]:
data['prediction'] = 0

In [ ]:
data['prediction'][-len(results.diff(1).y_pred):] = results.diff(1).y_pred

In [ ]:
data['signal'] = 0

In [ ]:
data['signal'][-len(results.diff(1).y_pred):] = [-1 if i <= 0 else 1 for i in results.diff(1).y_pred]

In [ ]:
data['ETH_diff'] = data['ETH'].diff(1)

In [ ]:
data

In [ ]:
data['signal_adj'] = data.apply(lambda t: 0 if abs(t['prediction']) < 0.02 else t['signal'], axis=1)

In [ ]:
data

In [ ]:
# Set the initial capital
initial_capital= float(10000.0)

In [ ]:
# Create a DataFrame `positions`
positions = pd.DataFrame(index=data.index).fillna(0.0)

In [ ]:
# Buy a 100 shares
positions['ETH'] = 10*data['signal_adj']   

In [ ]:
# Initialize the portfolio with value owned   
portfolio = positions.multiply(data['ETH'], axis=0)

In [ ]:
# Store the difference in shares owned 
pos_diff = positions.diff()

# Add `holdings` to portfolio
portfolio['holdings'] = (positions.multiply(data['ETH'], axis=0)).sum(axis=1)

# Add `cash` to portfolio
portfolio['cash'] = initial_capital - (pos_diff.multiply(data['ETH'], axis=0)).sum(axis=1).cumsum()   

# Add `total` to portfolio
portfolio['total'] = portfolio['cash'] + portfolio['holdings']

# Add `returns` to portfolio
portfolio['returns'] = portfolio['total'].pct_change()

# Print the first lines of `portfolio`
print(portfolio.head())

In [ ]:
# Print the last lines of `portfolio`
portfolio.tail() 